In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [23]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dependent=dataset[['charges']].values

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [25]:
import xgboost as xgb

In [26]:
xgb_reg=xgb.XGBRegressor()

In [27]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
#grid_search.fit(X_train, y_train)




In [28]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(xgb_reg,param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learnin...
                                    n_jobs=None, num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None,
                                    reg_alpha=None, ...),
             iid='warn', 

In [29]:
grid_pred=grid.predict(x_test)
grid_pred

array([12739.028 , 10352.542 , 45564.32  , 13823.025 ,  9286.334 ,
        4509.836 ,  2384.424 , 13183.99  ,  7351.125 ,  6187.795 ,
        6048.002 , 12303.582 ,  9497.648 ,  6669.917 , 19823.615 ,
       12471.052 , 14683.347 ,  4845.1616,  7012.518 , 32880.402 ,
       25588.047 , 15489.063 , 11802.942 , 26976.703 ,  3422.9016,
        6005.457 ,  5838.848 ,  8301.145 ,  4362.902 , 11247.32  ,
        7806.1826, 47147.113 , 15329.742 , 11953.06  , 17688.84  ,
        4944.8506, 13286.851 , 38442.348 , 39020.582 ,  3511.4   ,
        4904.802 ,  5207.5024, 19862.246 , 47689.773 , 37217.465 ,
        4558.747 , 12578.2705,  7132.0933,  5208.657 , 12524.195 ,
        4285.3086,  4144.631 , 26614.525 , 46230.293 , 11484.758 ,
        6997.8257,  5738.8574, 10261.804 , 10007.154 , 16655.314 ,
        2534.4631, 44013.53  , 17941.334 , 12525.45  , 13493.164 ,
       10186.191 , 32296.797 , 38316.29  ,  4011.7812,  9180.003 ,
       14939.863 , 12772.901 , 18023.83  , 15123.2   , 14123.6

In [30]:
from sklearn.metrics import r2_score
r2_score=r2_score(y_test,grid_pred)
r2_score

0.886199462783251

In [20]:
# Print the best hyperparameters and corresponding MSE score
print("Best Hyperparameters: ", grid.best_params_)
print("Best MSE Score: ", -grid.best_score_)

Best Hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 0.9}
Best MSE Score:  23322644.313994475


# Save model

In [21]:
import pickle
file_name="finalized_XGboost.sav"

pickle.dump(grid,open(file_name,"wb"))
loaded_model=pickle.load(open("finalized_XGboost.sav","rb"))
loaded_model


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learnin...
                                    n_jobs=None, num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None,
                                    reg_alpha=None, ...),
             iid='warn', 